In [51]:
# Prices are in €/MWh while production figures are in kWh it is not handled yet

import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB


def CalcRevenue(data):
    solutions  = list()
    for index,row in data.iterrows():
        
            hourlyRev = row['DA_PriceEUR'] * row['modProd']
            
            
            if row['modProd'] > row['actProd']:
                hourlyRev = hourlyRev - row['BalancingPriceUpEUR'] * (row['modProd']-row['actProd'])
            elif row['actProd'] > row['modProd']:
                hourlyRev = hourlyRev + row['BalancingPriceDownEUR'] * (row['actProd']-row['modProd'])
                
            solutions.append({row['HourDK']:hourlyRev})
    
    return solutions





# p_t_hat as decision variable
def OptimizationProblemEnergybid(prices_df):
    model = gp.Model()
    solutions  = list()
    capacity = 30000.0
    
    for index, row in prices_df.iterrows():
        
        p_t_hat = model.addVar(name="p_t_hat", lb=0, ub=capacity, vtype=GRB.INTEGER)
        z_down = model.addVar(name="z_down", vtype=GRB.CONTINUOUS, lb=0)
        z_up = model.addVar(name="z_up", vtype=GRB.CONTINUOUS, lb=0)
        
        condition_down = model.addVar(name="condition_down", vtype=GRB.BINARY)
        condition_up = model.addVar(name="condition_up", vtype=GRB.BINARY)
        
        model.addConstr(z_up <= (p_t_hat - row['p_t']) * condition_up)
        model.addConstr(z_up >= (p_t_hat - row['p_t']) * condition_up)
        model.addConstr(z_down <= (row['p_t'] - p_t_hat) * condition_down)
        model.addConstr(z_down >= (row['p_t'] - p_t_hat) * condition_down)
        model.addConstr(condition_up + condition_down == 1)
        
    
        #defining the constraints
        model.addConstr(-p_t_hat <= 0)
        model.addConstr(p_t_hat <= capacity)
        model.addConstr(-z_down <= 0)
        model.addConstr(-z_up <= 0)
        
        model.setObjective((row['DA_PriceEUR']*p_t_hat)+(row['BalancingPriceDownEUR']*z_down-row['BalancingPriceUpEUR']*z_up), GRB.MAXIMIZE)
        
        model.optimize()
    
        if model.status == GRB.OPTIMAL:
            print("Optimal solution found!")
            print(f"Objective Value: {model.ObjVal}")
            values = []
            values.append({ "Day-Ahead-Price": row['DA_PriceEUR'],
                            "BalancingPriceUp":row['BalancingPriceUpEUR'],
                            "BalancingPriceDown":row['BalancingPriceDownEUR'],
                            "p_t":row['p_t'],
                            "Revenue":model.ObjVal})
            for v in model.getVars():
                print(f"{v.varName}: {v.x}")
                values.append({v.varName:v.x})
        
            
            solutions.append({row['HourDK']:values})
                
        elif model.status == GRB.INFEASIBLE:
            print("Model is infeasible.")
            model.computeIIS()
            model.write("infeasible.ilp")  # Write IIS to a file for review
            for c in model.getConstrs():
                if c.IISConstr:
                    print(f"Infeasible constraint: {c.constrName}")
            
            
        elif model.status == GRB.UNBOUNDED:
            print("Model is unbounded.")
        else:
            print("Model status:", model.status)
            
        model.remove(model.getConstrs())
        model.remove(model.getVars())
        
    return solutions
        
def ObservationPeriod(Start_Observation, End_Observation, data):
    Start_Observation = pd.to_datetime(Start_Observation)
    End_Observation = pd.to_datetime(End_Observation)
    condition = (data["HourDK"]>= Start_Observation) & (data["HourDK"]< End_Observation)
    data  = data[condition]
    return data
        

In [82]:
prices_df = pd.read_csv('../Data assignment 1/prices_merged_df_output.csv', delimiter=',')

features_df = pd.read_csv('../Data assignment 1/Feature data.csv', delimiter=',')
features_df['AKI Kalby Active Power'] = -1 * features_df['AKI Kalby Active Power']
data = prices_df
data['actProd'] = features_df['AKI Kalby Active Power']
data["HourDK"] = pd.to_datetime(data["HourDK"])
data = ObservationPeriod("2022-01-05 10:00:00", "2022-01-05 20:00:00", data)

#Here is the interface to get the prediction timeseries
p_t1, p_t2, p_t3 = (np.random.uniform(0.0, 30000, size=(10, 1)) for _ in range(3))
p_t1, p_t2, p_t3 = (np.round(arr, 2) for arr in (p_t1, p_t2, p_t3))

data['LinReg'] = p_t1
data['NonLinReg'] = p_t2
data["ReguReg"] = p_t3






                    HourDK  BalancingPriceUpEUR  BalancingPriceDownEUR  \
count                 8760          8759.000000            8759.000000   
mean   2022-07-02 11:30:00           231.754458             184.503993   
min    2022-01-01 00:00:00             0.000000               0.000000   
25%    2022-04-02 05:45:00           109.626648              75.000000   
50%    2022-07-02 11:30:00           196.040329             162.000458   
75%    2022-10-01 17:15:00           320.005005             255.803902   
max    2022-12-31 23:00:00          1344.158325             871.000000   
std                    NaN           171.474017             147.136901   

       DA_PriceEUR      actProd  
count  8759.000000  7813.000000  
mean    210.128609  1380.265229  
min       0.000000   -48.208845  
25%     100.100000   163.352103  
50%     187.010000   796.908783  
75%     286.985000  2217.287513  
max     871.000000  5906.405585  
std     150.269703  1509.643752  
517


/var/folders/5y/2d5f2m356xd_kxc8krzn20q40000gn/T/ipykernel_11508/2192547491.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['LinReg'] = p_t1
/var/folders/5y/2d5f2m356xd_kxc8krzn20q40000gn/T/ipykernel_11508/2192547491.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NonLinReg'] = p_t2
/var/folders/5y/2d5f2m356xd_kxc8krzn20q40000gn/T/ipykernel_11508/2192547491.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [58]:
# Hourly Optimmization Problem
data['p_t'] = p_t1
solutions = OptimizationProblemEnergybid(data)
solutions

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.6.0 22G830)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x8c5874f9
Model has 4 quadratic constraints
Variable types: 2 continuous, 3 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+04]
  Objective range  [1e+02, 1e+02]
  Bounds range     [1e+00, 3e+04]
  RHS range        [1e+00, 3e+04]
Presolve removed 4 rows and 1 columns
Presolve time: 0.02s
Presolved: 4 rows, 7 columns, 11 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 2 continuous, 5 integer (2 binary)
Found heuristic solution: objective 2662277.1126

Root relaxation: interrupted, 0 iterations, 0.02 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.10 seconds (0.00 work un

/var/folders/5y/2d5f2m356xd_kxc8krzn20q40000gn/T/ipykernel_11508/1635347246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['p_t'] = p_t


Presolved: 4 rows, 7 columns, 11 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 2 continuous, 5 integer (2 binary)
Found heuristic solution: objective 1120951.8569

Root relaxation: interrupted, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 1.12095e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.120951856887e+06, best bound 1.120951860300e+06, gap 0.0000%
Optimal solution found!
Objective Value: 1120951.8568874667
p_t_hat: 8493.0
z_down: 0.9899999999761349
z_up: 0.0
condition_down: 1.0
condition_up: -0.0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.6.0 22G830)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 5 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x88fa8977
Model has 16 

[{Timestamp('2022-01-05 10:00:00'): [{'Day-Ahead-Price': 135.05,
    'BalancingPriceUp': 148.995758,
    'BalancingPriceDown': 135.046509,
    'p_t': 19713.27,
    'Revenue': 2662277.1125574266},
   {'p_t_hat': 19713.0},
   {'z_down': 0.2699999999713327},
   {'z_up': 0.0},
   {'condition_down': 1.0},
   {'condition_up': -0.0}]},
 {Timestamp('2022-01-05 11:00:00'): [{'Day-Ahead-Price': 129.99,
    'BalancingPriceUp': 148.995758,
    'BalancingPriceDown': 129.986282,
    'p_t': 28752.96,
    'Revenue': 3737597.266830729},
   {'p_t_hat': 28752.0},
   {'z_down': 0.9600000000646105},
   {'z_up': 0.0},
   {'condition_down': 1.0},
   {'condition_up': -0.0}]},
 {Timestamp('2022-01-05 12:00:00'): [{'Day-Ahead-Price': 131.97,
    'BalancingPriceUp': 148.995758,
    'BalancingPriceDown': 131.966553,
    'p_t': 8493.99,
    'Revenue': 1120951.8568874667},
   {'p_t_hat': 8493.0},
   {'z_down': 0.9899999999761349},
   {'z_up': 0.0},
   {'condition_down': 1.0},
   {'condition_up': -0.0}]},
 {Timestam